# Spark Apache (семинары)

## Урок 3. Инструменты работы и визуализации ч.2

Условие: есть набор данных о продажах продуктов с информацией о дате продаж, категории продукта, количестве и выручке от продаж.

Используя Apache Spark, загрузите предоставленный набор данных в DataFrame (пример данных ниже).

```
("2023-11-20", "Electronics", 100, 12000),
("2023-11-21", "Electronics", 110, 13000),
("2023-11-22", "Electronics", 105, 12500),
("2023-11-20", "Clothing", 300, 15000),
("2023-11-21", "Clothing", 280, 14000),
("2023-11-22", "Clothing", 320, 16000),
("2023-11-20", "Books", 150, 9000),
("2023-11-21", "Books", 200, 12000),
("2023-11-22", "Books", 180, 10000)
```

Столбцы: "date", "category", "quantity", "revenue".

С использованием оконных функций, рассчитайте среднее выручки от продаж для каждой категории продукта.

Примените операцию pivot для того, чтобы преобразовать полученные данные таким образом, чтобы в качестве строк были категории продуктов, в качестве столбцов были дни, а значениями были средние значения выручки от продаж за соответствующий день

In [1]:
%pip install pyspark django >> None

In [17]:
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType, DateType
from pyspark.sql.functions import avg, col
from pyspark.sql.window import Window
import pyspark.sql.functions as fn

# Создание Spark сессии
spark = SparkSession.builder.appName("HW_3").getOrCreate()

# Исходные данные
data = [("2023-11-20", "Electronics", 100, 12000),
        ("2023-11-21", "Electronics", 110, 13000),
        ("2023-11-22", "Electronics", 105, 12500),
        ("2023-11-20", "Clothing", 300, 15000),
        ("2023-11-21", "Clothing", 280, 14000),
        ("2023-11-22", "Clothing", 320, 16000),
        ("2023-11-20", "Books", 150, 9000),
        ("2023-11-21", "Books", 200, 12000),
        ("2023-11-22", "Books", 180, 10000)]

# Создание DataFrame
columns = ["date", "category", "quantity", "revenue"]
df = spark.createDataFrame(data, columns)

# Преобразование столбца "date" в тип Date
df = df.withColumn("date", col("date").cast(DateType()))

# Определение оконной спецификации
window_spec = Window.partitionBy("category")

# Оконная функция: вычисление средней выручки от продаж для каждой категории продукта
avg_revenue = fn.avg("revenue").over(window_spec)

# Применение оконной функции и приведение к double
result_df = df.withColumn("avg_revenue", avg_revenue.cast(DoubleType()))

# Отображение результатов: используем distinct() для исключения повторов
result_df.select(["category", "avg_revenue"]).distinct().show()

+-----------+------------------+
|   category|       avg_revenue|
+-----------+------------------+
|      Books|10333.333333333334|
|   Clothing|           15000.0|
|Electronics|           12500.0|
+-----------+------------------+



In [18]:
# Определение при помощи функции pivot средних значений выручки от продаж по категориям за соответствующий день
pivot_df = df.groupBy("category").pivot("date").agg(fn.avg("revenue")).orderBy("category")

# Вывод результатов
pivot_df.show()

# Завершение работы SparkSession
spark.stop()

+-----------+----------+----------+----------+
|   category|2023-11-20|2023-11-21|2023-11-22|
+-----------+----------+----------+----------+
|      Books|    9000.0|   12000.0|   10000.0|
|   Clothing|   15000.0|   14000.0|   16000.0|
|Electronics|   12000.0|   13000.0|   12500.0|
+-----------+----------+----------+----------+

